In [2]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cpu')

In [4]:
training_data = pd.read_csv("../../datasets/FashionMnist/fashion-mnist_train.csv")
testing_data = pd.read_csv("../../datasets/FashionMnist/fashion-mnist_test.csv")
xtrain = training_data.iloc[:,1:]
ytrain = training_data.iloc[:, 0]

xtest = testing_data.iloc[:,1:]
ytest = testing_data.iloc[:,0]

ytrain = ytrain.to_numpy().reshape(-1,1).astype('float32') 
ytest = ytest.to_numpy().reshape(-1,1).astype('float32') 

In [5]:
temp = np.zeros([len(ytrain),10],dtype="float32")
for i, repl in enumerate(ytrain):
    index = repl[0].astype('int')
    temp[i,index] = repl[0]

ytrain = temp


In [6]:
temp = np.zeros([len(ytest),10],dtype="float32")
for i, repl in enumerate(ytest):
    index = repl[0].astype('int')
    temp[i,index] = repl[0]

ytest = temp

In [7]:
mean = np.nanmean(pd.concat([xtrain,xtest]))/255
std = pd.concat([xtrain,xtest]).values.std()/255

In [8]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

In [9]:
class fashionMnistData(torch.utils.data.Dataset):
    def __init__(self,features: pd.Series, labels: pd.Series, transformer=None):
        self.features = features
        self.labels = labels
        self.transform = transformer

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        image = self.features.iloc[idx].values.reshape(28, 28).astype('float32')
        label = self.labels.iloc[idx] if isinstance(self.labels, pd.Series) else self.labels[idx]
        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label).to(device)
        return image, label

In [10]:
class fashionMnistClassification(nn.Module):
    def __init__(self, input, hid1, hid2, hid3, output, activation=nn.ReLU()):
        super(fashionMnistClassification, self).__init__()
        self.sequence = nn.Sequential(
            nn.Linear(input, hid1),
            activation,
            nn.Linear(hid1, hid2),
            activation,
            nn.Linear(hid2, hid3),
            activation,
            nn.Linear(hid3, output),
            activation
        )

    def forward(self, x):
        output = self.sequence(x)
        return output

In [11]:
# Hyperparams
epochs = 2
batchsize = 100
lrate = 0.0001
inputsz = xtrain.shape[1]
hid1 = 32
hid2 = 32
hid3 = 32
output = 10


In [12]:
# Initializing dataloaders

traindb = fashionMnistData(xtrain, ytrain, transformer)
testdb = fashionMnistData(xtest, ytest, transformer)

trainloader = torch.utils.data.DataLoader(traindb, batch_size=batchsize, shuffle=True)
testloader = torch.utils.data.DataLoader(testdb, batch_size=batchsize, shuffle=True)


In [13]:
model = fashionMnistClassification(inputsz,hid1,hid2,hid3,output,nn.ReLU()).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lrate)

In [14]:
# training the model
model.train()
loss = 0
for epoch in range(epochs):
    for images, labels in trainloader:
        images = images.reshape(-1,28*28).to(device)
        
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
print(f"entropy loss: {loss.item():.4f}")


entropy loss: 2.6317


In [15]:
# Compare
import torch.nn.functional as F
import random
index = random.randrange(0,100)

model.eval()
with torch.no_grad():
    for image, label in testloader:
        image = image.reshape(-1,28*28).to(device)
        output = model(image)
        probabilities = F.softmax(output, dim=1)
        _, predicted = torch.max(probabilities, 1)
        print(f"prediction: {predicted[index]}\nActual: {label[index]}\nindex: {index}")
        break

prediction: 7
Actual: tensor([0., 0., 0., 0., 0., 0., 0., 7., 0., 0.])
index: 63
